# Author Giovambattista Vieri
# (c) Giovambattista Vieri 2020 all rights reserved
# License GNU AFFERO GPL

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
import time
from ipywidgets import widgets

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome('chromedriver',options=chrome_options)




In [ ]:
from ipywidgets import widgets

from pprint import pprint

lbl1=widgets.Label('Please insert url complete with http/https')
txt1=widgets.Text(placeholder='insert valid url')
bttn1=widgets.Button(description='proceed')
def proceed(b):
    url=str(txt1.value)
    browser.get(url)
    time.sleep(2)
    print (f"===== cookie/s for {url} =====\n")
    cookies = browser.get_cookies()
    links=browser.find_elements_by_tag_name('link')

    for cookie in cookies:
        print(cookie)
    
    print(f"+++++ links/s for {url} ++++++\n")
    for link in links:
        href = link.get_attribute('href')
        print (href)
    print(f"-----  end for {url} -----\n") 
bttn1.on_click(proceed)
display(lbl1,txt1,bttn1)
